In [6]:
from pyannote.audio import Pipeline
import torch
import os

api_key = os.getenv("HUGGING_TOKEN")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=api_key
)

if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print("cuda is available")
else:
    print('cuda is not available')

/Users/yoohoyeong/Desktop/AI_Practic/venv/lib/python3.12/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


cuda is not available


In [7]:
diarization = pipeline("../audio/싼기타_비싼기타.mp3")

with open("../audio/싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
  diarization.write_rttm(rttm)

/Users/yoohoyeong/Desktop/AI_Practic/venv/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/yoohoyeong/Desktop/AI_Practic/venv/lib/python3.12/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
/Users/yoohoyeong/Desktop/

In [8]:
import pandas as pd
rttm_path = "../audio/싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
  rttm_path,
  sep=' ',
  header=None,
  names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4']
)

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN


In [9]:
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204
...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268


In [13]:
df_rttm["number"] = None
df_rttm.at[0, "number"] = 0

for i in range(1, len(df_rttm)):
  if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
    df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
  else:
    df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,0
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,0
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,0
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,0
...,...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451,31
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231,32
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420,33
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268,33


In [15]:
df_rttm_grouped = df_rttm.groupby("number").agg(
  start=pd.NamedAgg(column='start', aggfunc='min'),
  end=pd.NamedAgg(column='end', aggfunc="max"),
  speaker_id=pd.NamedAgg(column='speaker_id', aggfunc='first')
)

display(df_rttm_grouped)

,start,end,speaker_id
number,,,
0,0.993,30.204,SPEAKER_01
1,32.414,42.708,SPEAKER_00
2,41.645,44.024,SPEAKER_01
3,45.813,67.109,SPEAKER_00
4,67.227,82.786,SPEAKER_01
5,84.659,102.564,SPEAKER_00
6,103.492,117.532,SPEAKER_01
7,119.759,138.676,SPEAKER_00
8,139.351,168.967,SPEAKER_01


In [ ]:
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)